In [1]:
import xarray as xr
from pathlib import Path
import copy
import pandas as pd
import json
import os
from datetime import datetime, timedelta
from osgeo import gdal 
import numpy as np

base_path = Path('/fs/site6/eccc/cmd/w/sbf000')


In [34]:
import pandas as pd
from datetime import datetime, timedelta

def round_date_to_seconds(d):
    return pd.to_datetime((d.replace(second=d.second, microsecond=0, minute=d.minute, hour=d.hour) + timedelta(seconds=round(d.microsecond/1000000))).strftime('%Y%m%dT%H%M%SZ'))

file = '199801f.txt.gz'

def to_cg(s):
    return 'cloud' if s == 1 else 'ground'

def to_epoch(c):
    return c.strftime('%s')

df = pd.read_csv(str(base_path / file), sep=' ', index_col=False, header=0, names=['date', 'time','lat', 'lon','event_strenght','chi_squared_value', 'semi_major_axis', 'semi_minor_axis','angle_of_the_confidence', 'cloud_or_ground_status'])
df["event_time"] = pd.to_datetime(df['date'].astype(str) +"T"+ df["time"]+"Z")
df['time'] = df["event_time"].map(round_date_to_seconds)
df["event_time"] = df["event_time"].astype(str)
# df["event_time"] = pd.to_datetime(df['date'].astype(str) +"T"+ df["time"] + 'Z')
df["time"] = df["time"].map(to_epoch)
df = df.drop(columns=['date'])
df['cloud_or_ground_status'] = df['cloud_or_ground_status'].map(to_cg)
df['type_of_data'] = 'flash'
df.to_csv('/199801f.csv', index=False, header=False)

In [52]:

def round_date_to_minutes(d):
    return pd.to_datetime((d.replace(second=0, microsecond=0, minute=0, hour=d.hour) + timedelta(hours=d.minute//30)).strftime('%Y%m%dT%H'))

file = '199801f.txt.gz'
df = pd.read_csv(str(base_path / file), sep=' ', index_col=False, header=0, names=['date', 'time','lat', 'lon','event_strenght','chi_squared_value', 'semi_major_axis', 'semi_minor_axis','angle_of_the_confidence', 'cloud_or_ground_status'])
df["event_time"] = pd.to_datetime(df['date'].astype(str) +"T"+ df["time"])
df = df.drop(columns=['date','time'])
# df['valid_time'] = df["event_time"].map(round_date_to_minutes)
df = df.sort_values(['valid_time', 'lat','lon'], ascending = [True, False, True])
# timedelta.
df['diff'] = (df['event_time'] - df['valid_time'])
def to_micro(d):
    return d.microseconds
df['diff'] = df['diff'].map(to_micro)
display(df)
# for i in df.index:
#     df.at[i,'class'] = ','.join([df.lat.astype('str'),df.lon.astype('str')])
# df

groups = df.groupby('valid_time')
counter = 0
path = '/home/sbf000/ss6/lightning/'
os.chdir(path)

# for valid_time, sub_df in groups:
valid_time = ''
for col in ['eventStrenght','ChiSquaredValue', 'SemiMajorAxis', 'SemiMinorAxis','AngleOfTheConfidence', 'CloudOrGroundStatus']:
    # file = ''.join([valid_time.strftime('%Y%m%dT%H%M%SZ'),col])
    file = ''.join([valid_time,col])
    csv_file = ''.join([file,'.csv'])
    if os.path.exists(csv_file): 
        os.remove(csv_file) 

    df[['lat','lon', col]].to_csv(csv_file, index = False)

    vrt_file = ''.join([file,'.vrt'])
    if os.path.exists(vrt_file): 
        os.remove(vrt_file) 

    with open(vrt_file, "w") as f:
        # !!! here you have to put the information that should be written to the VRT file. YouTube doesn't allow angled brackets in the description, so I can't put that here, but you can copy the necessary lines from the GDAL webpage: https://gdal.org/programs/gdal_grid.h... 
        vrt_contents = f'<OGRVRTDataSource> \n\
            <OGRVRTLayer name="{col}"> \n\
                <SrcDataSource>{csv_file}</SrcDataSource> \n\
                <LayerSRS>EPSG:4326</LayerSRS> \n\
                <GeometryType>wkbPoint</GeometryType> \n\
                <GeometryField encoding="PointFromColumns" x="lon" y="lat" z="{col}"/> \n\
            </OGRVRTLayer> \n\
        </OGRVRTDataSource>\n'
        f.write(vrt_contents)

    tif_file1 = ''.join([file,'1.tif'])
    if os.path.exists(tif_file1): 
        os.remove(tif_file1)     
    
    tif_file2 = ''.join([file,'2.tif'])
    if os.path.exists(tif_file2): 
        os.remove(tif_file2)     

    # r = gdal.Rasterize(tif_file1, vrt_file, outputSRS = "EPSG:4326", xRes = 0.03, yRes = -0.03, attribute = col, noData = np.nan) 
    # r = None     
    g = gdal.Grid(tif_file2, vrt_file, outputSRS = "EPSG:4326")     




,lat,lon,eventStrenght,ChiSquaredValue,SemiMajorAxis,SemiMinorAxis,AngleOfTheConfidence,CloudOrGroundStatus,event_time,valid_time,diff
0,37.0588,-68.6269,175.1,8.8,2.1,0.5,119,1,1998-01-01 01:36:01.402053700,1998-01-01 02:00:00,402053
2,35.7650,-72.8419,31.3,0.4,2.2,0.5,142,1,1998-01-01 03:57:43.857789100,1998-01-01 04:00:00,857789
3,35.7310,-72.9569,-36.5,5.2,1.0,0.4,124,1,1998-01-01 03:57:44.028975418,1998-01-01 04:00:00,28975
4,35.7200,-71.5627,57.3,6.7,0.9,0.3,134,1,1998-01-01 04:01:06.282420872,1998-01-01 04:00:00,282420
1,35.6156,-72.6034,165.9,12.0,3.3,0.6,126,1,1998-01-01 03:57:43.843714110,1998-01-01 04:00:00,843714
...,...,...,...,...,...,...,...,...,...,...,...
37194,35.0914,-97.7602,-10.5,1.0,0.4,0.3,143,4,1998-01-31 23:49:26.314892100,1998-02-01 00:00:00,314892
37181,35.0731,-97.6800,29.3,1.4,0.3,0.2,100,1,1998-01-31 23:44:46.282819938,1998-02-01 00:00:00,282819
37190,35.0658,-97.7432,-13.5,1.1,0.4,0.3,142,1,1998-01-31 23:48:31.560917700,1998-02-01 00:00:00,560917
37179,35.0440,-97.7112,-12.9,1.2,0.4,0.3,97,1,1998-01-31 23:43:54.531273600,1998-02-01 00:00:00,531273


In [ ]:
if os.path.exists("uneven.csv"): 
    os.remove("uneven.csv") 

df[['lat','lon','event_strenght']].to_csv("uneven.csv",index = False)

if os.path.exists("uneven.vrt"): 
    os.remove("uneven.vrt") 

with open("uneven.vrt", "w") as f:
    # !!! here you have to put the information that should be written to the VRT file. YouTube doesn't allow angled brackets in the description, so I can't put that here, but you can copy the necessary lines from the GDAL webpage: https://gdal.org/programs/gdal_grid.h... 
    vrt_contents = '<OGRVRTDataSource> \
        <OGRVRTLayer name="uneven"> \
            <SrcDataSource>uneven.csv</SrcDataSource> \
            <GeometryType>wkbPoint</GeometryType> \
            <GeometryField encoding="PointFromColumns" x="lon" y="lat" z="event_strenght"/> \
        </OGRVRTLayer> \
    </OGRVRTDataSource>'
    f.write(vrt_contents)

r = gdal.Rasterize("/home/sbf000/ss6/uneven.tif", "uneven.vrt", outputSRS = "EPSG:4326", xRes = 0.03, yRes = -0.03, attribute = "event_strenght", noData = np.nan) 
r = None 
g = gdal.Grid("/home/sbf000/ss6/unevenInt.tif", "uneven.vrt", outputSRS = "EPSG:4326") 
g = None    


In [17]:
ls -lah /home/sbf000/ss6/

total 23M
drwxr-xr-x 14 sbf000  eccc_cmdw  32K Jun 10 14:54 ./
drwxrwxr-x 24 sadm800 eccc_cmdw 4.0K May 22 16:44 ../
-r--r--r--  1 sbf000  eccc_cmdw 865K Jun  9 20:12 199801f.txt.gz
-rw-r--r--  1 sbf000  eccc_cmdw 1.2K May  9 20:10 display_operation_database.py
-rw-r--r--  1 sbf000  eccc_cmdw 2.6K May 13 14:43 grib_database.csv
-rw-r--r--  1 sbf000  eccc_cmdw 1.9K Jun  9 20:19 lightning_data_readme.txt
drwxr-xr-x  3 sbf000  eccc_cmdw 4.0K May 18 10:38 logs/
-rw-r--r--  1 sbf000  eccc_cmdw 143K May 11 19:48 monresultat_timeintminmax.std
drwxr-xr-x  2 sbf000  eccc_cmdw 4.0K May 14 20:15 netcdf/
drwxr-xr-x  2 sbf000  eccc_cmdw 4.0K May 15 21:16 operation.forecasts.gdsps.netcdf/
drwxr-xr-x  2 sbf000  eccc_cmdw 4.0M Jun  8 18:12 operation.forecasts.lamgrib2.hrdps_national.continental/
drwxr-xr-x  3 sbf000  eccc_cmdw  32K May 20 12:19 operation.forecasts.lam.nat.pres/
drwxr-xr-x  2 sbf000  eccc_cmdw 4.0K Jun  6 17:37 operation.grib2.gdps/
drwxr-xr-x  2 sbf000  eccc_cmdw 4.0K Jun  6 17:15 ope

In [ ]:
# https://www.youtube.com/watch?v=zLNLG0j13Cw

from osgeo import gdal 
import pandas as pd 
import numpy as np 
import os 
import matplotlib.pyplot as plt 
ds = gdal.Open("dem.tif") 
# TIFF to CSV 

# first option 
xyz = gdal.Translate("dem.xyz", ds) 
xyz = None 
df = pd.read_csv("dem.xyz", sep = " ", header = None) 
df.columns = ["x","y", "value"] 
df.to_csv("dem.csv", index = False) 

# second option 
ar = ds.GetRasterBand(1).ReadAsArray() 
flat = ar.flatten() 
gt = ds.GetGeoTransform() 
res = gt[1] 
xmin = gt[0] 
ymax = gt[3] 
xsize = ds.RasterXSize 
ysize = ds.RasterYSize 
xstart = xmin +res/2 
ystart = ymax - res/2 
ds = None 
x = np.arange(xstart, xstart+xsize*res, res) 
y = np.arange(ystart, ystart-ysize*res, -res) 
x = np.tile(x, ysize) 
y = np.repeat(y, xsize) 
dfn = pd.DataFrame({"x":x, "y":y, "value":flat}) 
# CSV to TIFF 
# first option 
dfn.to_csv("dfn.xyz", index = False, header = None, sep = " ") 
demn = gdal.Translate("demn.tif", "dfn.xyz", outputSRS = "EPSG:32719") 
demn = None 

def toTIFF(dfn, name): 
    dfn.to_csv(name+".xyz", index = False, header = None, sep = " ") 
    demn = gdal.Translate(name+".tif", name+".xyz", outputSRS = "EPSG:32719", xRes = res, yRes = -res) 
    demn = None 

shuffle = dfn.sample(frac = 1) 
shuffle = shuffle.sort_values(by = ["y", "x"], ascending = [False, True]) 

toTIFF(shuffle, "shuffle") 
sample = dfn.sample(frac = 0.1) 
sample = sample.sort_values(by = ["y", "x"], ascending = [False, True]) 

toTIFF(sample, "sample") 
uneven = sample.copy() 
uneven.x = uneven.x + np.random.randint(6, size = len(uneven)) 
uneven.y = uneven.y + np.random.randint(6, size = len(uneven)) 

# toTIFF(uneven, "uneven") 
# not working 
# second option 
uneven.to_csv("uneven.csv", index = False) 

if os.path.exists("uneven.vrt"): 
    os.remove("uneven.vrt") 

with open("uneven.vrt", "w") as f:
    # !!! here you have to put the information that should be written to the VRT file. YouTube doesn't allow angled brackets in the description, so I can't put that here, but you can copy the necessary lines from the GDAL webpage: https://gdal.org/programs/gdal_grid.h... 
    vrt_contents = '<OGRVRTDataSource> \
        <OGRVRTLayer name="dem"> \
            <SrcDataSource>dem.csv</SrcDataSource> \
            <GeometryType>wkbPoint</GeometryType> \
            <GeometryField encoding="PointFromColumns" x="Easting" y="Northing" z="Elevation"/> \
        </OGRVRTLayer> \
    </OGRVRTDataSource>'
    f.write(vrt_contents) 


r = gdal.Rasterize("uneven.tif", "uneven.vrt", outputSRS = "EPSG:32719", xRes = res, yRes = -res, attribute = "value", noData = np.nan) 
r = None 
g = gdal.Grid("unevenInt.tif", "uneven.vrt", outputSRS = "EPSG:32719") 
g = None



In [1]:
from dask.distributed import Client

client = Client(n_workers=6)
client

distributed.diskutils - INFO - Found stale lock file and directory '/fs/homeu2/eccc/cmd/cmdw/sbf000/src/pairs/notebooks/dask-worker-space/worker-smnhpqqg', purging
distributed.diskutils - INFO - Found stale lock file and directory '/fs/homeu2/eccc/cmd/cmdw/sbf000/src/pairs/notebooks/dask-worker-space/worker-6ffwq31u', purging
distributed.diskutils - INFO - Found stale lock file and directory '/fs/homeu2/eccc/cmd/cmdw/sbf000/src/pairs/notebooks/dask-worker-space/worker-lx05skqe', purging
distributed.diskutils - INFO - Found stale lock file and directory '/fs/homeu2/eccc/cmd/cmdw/sbf000/src/pairs/notebooks/dask-worker-space/worker-xptaevks', purging
distributed.diskutils - INFO - Found stale lock file and directory '/fs/homeu2/eccc/cmd/cmdw/sbf000/src/pairs/notebooks/dask-worker-space/worker-blb2_hl_', purging
distributed.diskutils - INFO - Found stale lock file and directory '/fs/homeu2/eccc/cmd/cmdw/sbf000/src/pairs/notebooks/dask-worker-space/worker-jleslt4l', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 6
Total threads: 48,Total memory: 187.36 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40041,Workers: 6
Dashboard: http://127.0.0.1:8787/status,Total threads: 48
Started: Just now,Total memory: 187.36 GiB
Comm: tcp://127.0.0.1:47815,Total threads: 8
Dashboard: http://127.0.0.1:39107/status,Memory: 31.23 GiB
Nanny: tcp://127.0.0.1:34391,


In [29]:
from glob import glob
import pandas as pd
import numpy as np
import multiprocessing
from pathlib import Path

ffiles = glob(r'/fs/site5/eccc/cmd/w/sbf000/lightning_data/*f.txt.gz')
sfiles = glob(r'/fs/site5/eccc/cmd/w/sbf000/lightning_data/*s.txt.gz')
# print(len(ffiles))
# print(len(sfiles))
files = np.sort(ffiles + sfiles)
# print(len(files))

manager = multiprocessing.Manager()
manager.list()

lat_mins = manager.list()
lat_maxs = manager.list()
lon_mins = manager.list()
lon_maxs = manager.list()

def process(f):
    ltype = Path(f).stem[6]
    print(f'Processing {f}')
    if ltype =='f':
        df = pd.read_csv(f, sep=' ', names=['date', 'time','lat', 'lon','event_strenght','chi_squared_value', 'semi_major_axis', 'semi_minor_axis','angle_of_the_confidence','multiplicity', 'cloud_or_ground_status'], index_col=False)
    else:
        df = pd.read_csv(f, sep=' ', names=['date', 'time','lat', 'lon','event_strenght','chi_squared_value', 'semi_major_axis', 'semi_minor_axis','angle_of_the_confidence', 'cloud_or_ground_status'], index_col=False)
    lat_mins.append(df.lat.min())
    lat_maxs.append(df.lat.max())
    lon_mins.append(df.lon.min())
    lon_maxs.append(df.lon.max())

with multiprocessing.Pool(1) as p:
    p.map(process,files[-2:])

print(np.min(lat_mins))
print(np.max(lat_maxs))
print(np.min(lon_mins))
print(np.max(lon_maxs))

Processing /fs/site5/eccc/cmd/w/sbf000/lightning_data/202204f.txt.gz
Processing /fs/site5/eccc/cmd/w/sbf000/lightning_data/202204s.txt.gz
35.0
60.0033
-149.0505
-47.3222
